# Further Data Analysis Script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import shape, Polygon, Point
from libpysal.weights import Queen
from libpysal.weights.spatial_lag import lag_spatial

In [64]:
# read tracts data
PA_Tracts = gpd.read_file('/content/drive/MyDrive/data/PA_Tracts.geojson')

In [ ]:
# read cdc data
PA_Chronic = pd.read_csv('/content/drive/MyDrive/data/PA_Chronic.csv')
PA_HRB = pd.read_csv('/content/drive/MyDrive/data/PA_HRB.csv')

In [ ]:
# process cdc data
PA_Chronic = PA_Chronic[["CountyName", "LocationName", "Asthma", "TotalPopulation", "TotalPop18plus", "COP"]]
PA_HRB = PA_HRB[["CountyName", "LocationName", "Smoking", "Drinking", "Short_Sleep", "Physical_Activity"]]

In [ ]:
# read tobacco data
PA_Retailers = pd.read_csv('/content/drive/MyDrive/data/PA_Retailers.csv')

In [ ]:
# process retailers data
PA_Retailers['geometry'] = PA_Retailers.apply(
    lambda row: Point(row['lon'], row['lat']), axis=1
)

PA_Retailers = gpd.GeoDataFrame(
    PA_Retailers,
    geometry='geometry',
    crs='EPSG:4326'
)

In [ ]:
# count retailers
tracts_with_retailers = gpd.sjoin(PA_Tracts, PA_Retailers, how="left", predicate="intersects")
tracts_summary = (
    tracts_with_retailers.groupby("GEOID")
    .size()
    .reset_index(name="total_retailers")
)

tracts_with_retailers= PA_Tracts.merge(tracts_summary, on="GEOID", how="left")
tracts_with_retailers["total_retailers"] = tracts_with_retailers["total_retailers"].fillna(0)

# compute density
density = tracts_with_retailers.to_crs(epsg=3857)
density['area_km2'] = density.geometry.area / 1e6  # Convert m² to km²
density ['density'] = density ['total_retailers'] / density ['area_km2']

,GEOID,geometry,total_retailers,area_km2,density
0,42001030101,"POLYGON ((-8588920.357 4874186.492, -8588818.5...",3,95.080052,0.031552
1,42001030103,"POLYGON ((-8579678.266 4862280.523, -8579641.3...",1,31.775960,0.031470
2,42001030104,"POLYGON ((-8582635.361 4858365.994, -8582454.6...",7,86.167356,0.081237
3,42001030200,"POLYGON ((-8599802.938 4869004.417, -8599007.8...",8,207.033802,0.038641
4,42001030300,"POLYGON ((-8618506.453 4863224.423, -8618215.5...",5,191.098396,0.026165
...,...,...,...,...,...
3441,42133023902,"POLYGON ((-8526772.659 4837173.636, -8526669.9...",1,98.319324,0.010171
3442,42133023903,"POLYGON ((-8532719.338 4831745.365, -8532693.1...",8,31.425125,0.254573
3443,42133023904,"POLYGON ((-8535019.091 4834550.597, -8534977.0...",4,89.787440,0.044550
3444,42133024001,"POLYGON ((-8506177.304 4828656.467, -8506121.8...",11,131.697087,0.083525


In [ ]:
# Create spatial weights matrix based on tract geometries
weights = Queen.from_dataframe(density)

# Compute average density of neighbors using lag_spatial
density['neighbor_avg_density'] = lag_spatial(
    weights, density['density']
)

<ipython-input-61-5721ec2f458d>:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  weights = Queen.from_dataframe(density)


In [ ]:
PA_Retailers = density.drop(columns=["area_km2"])
PA_Retailers.head(5)

,GEOID,geometry,total_retailers,density,neighbor_avg_density
0,42001030101,"POLYGON ((-8588920.357 4874186.492, -8588818.5...",3,0.031552,0.227339
1,42001030103,"POLYGON ((-8579678.266 4862280.523, -8579641.3...",1,0.031470,0.158547
2,42001030104,"POLYGON ((-8582635.361 4858365.994, -8582454.6...",7,0.081237,1.671000
3,42001030200,"POLYGON ((-8599802.938 4869004.417, -8599007.8...",8,0.038641,0.413897
4,42001030300,"POLYGON ((-8618506.453 4863224.423, -8618215.5...",5,0.026165,0.216389


In [ ]:
# read census data
PA_Census = pd.read_csv('/content/drive/MyDrive/data/PA_Census_Data.csv')

In [ ]:
PA_Census.head(5)

,GEOID,minority,aging,disability,geometry
0,42125775200,0.211840,0.215264,0.287671,"POLYGON ((-79.876586 40.177549, -79.876234 40...."
1,42125775300,0.395709,0.131506,0.273739,"POLYGON ((-79.879294 40.174857, -79.878454 40...."
2,42125782700,0.048652,0.314267,0.302433,"POLYGON ((-79.913564 40.153257, -79.913332 40...."
3,42125783200,0.167610,0.160644,0.229430,"POLYGON ((-79.911421 40.144556, -79.909319 40...."
4,42125783300,0.167792,0.166234,0.159481,"POLYGON ((-79.906306 40.137474, -79.906219 40...."


In [ ]:
# read land cover data
folder_path = '/content/drive/MyDrive/data/PA_Landcover/'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

landcover = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    first_word = file.split()[0]
    df = pd.read_csv(file_path)
    df['source_file'] = first_word
    df = df.drop(columns=["system:index", ".geo"])
    landcover.append(df)


In [116]:
# combine landcover data
merged_df= pd.merge(landcover[0], landcover[1], on="geoid", how="left")
merged_df = merged_df.rename(
    columns={
        "total_landcover": "forest_total",
        "neighbor_landcover": "forest_neighbor",
        "sum": "wetland_total",
    }
)
merged_df = merged_df.drop(columns=["source_file_x", "source_file_y"])
merged_df = pd.merge(merged_df, landcover[2], on="geoid", how="left")
merged_df = merged_df.rename(
    columns={
        "total_landcover": "Hdensity_total",
        "neighbor_landcover": "Hdensity_neighbor"    }
)
merged_df = merged_df.drop(columns=["source_file"])
merged_df = pd.merge(merged_df, landcover[3], on="geoid", how="left")
merged_df = merged_df.rename(
    columns={
        "sum": "Ldensity_total"    }
)
merged_df = merged_df.drop(columns=["source_file"])
merged_df = pd.merge(merged_df, landcover[4], on="geoid", how="left")
merged_df = merged_df.rename(
    columns={
        "sum": "grasses_total"    }
)
merged_df = merged_df.drop(columns=["source_file"])
merged_df = pd.merge(merged_df, landcover[5], on="geoid", how="left")
merged_df = merged_df.rename(
    columns={
        "sum": "water_total"    }
)
PA_Landcover = merged_df.drop(columns=["source_file"])

In [117]:
# some metrics with landcover data
PA_Landcover['forest_pct'] = PA_Landcover['forest_neighbor'] / PA_Landcover['forest_total']
PA_Landcover['Hdensity_pct'] = PA_Landcover['Hdensity_neighbor'] / PA_Landcover['Hdensity_total']
PA_Landcover = PA_Landcover.drop(columns=["forest_neighbor", "Hdensity_neighbor"])

In [122]:
PA_Landcover.head(5)

,geoid,forest_total,wetland_total,Hdensity_total,Ldensity_total,grasses_total,water_total,forest_pct,Hdensity_pct
0,42001030101,14626.749020,2675.717647,817.870588,5868.972549,31291.925490,288.745098,0.470716,0.084107
1,42001030103,5035.964706,408.607843,283.894118,3644.113725,11254.184314,879.866667,0.407744,0.042269
2,42001030104,10099.086275,2142.858824,1041.843137,6041.019608,40245.227451,405.870588,0.354698,0.032345
3,42001030200,34923.054902,5905.745098,2194.835294,13370.447059,80858.125490,214.356863,0.429251,0.127547
4,42001030300,57866.054902,1336.996078,1100.000000,9721.752941,40909.298039,348.709804,0.718114,0.203636


In [65]:
# read in landsat data
folder_path = '/content/drive/MyDrive/data/PA_Landsat/'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

landsat = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    first_word = file.split()[0]
    df = pd.read_csv(file_path)
    df['source_file'] = first_word
    df = df.drop(columns=["system:index", ".geo"])
    landsat.append(df)

In [95]:
merged_df2= pd.merge(landsat[0], landsat[1], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean_x": "lst_winter",
        "mean_y": "ndvi_fall",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file_x", "source_file_y"])
merged_df2 = pd.merge(merged_df2, landsat[2], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "evi_spring",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[3], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "evi_fall",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[4], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "savi_fall",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[5], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "savi_winter",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[6], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "savi_summer",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[7], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "savi_spring",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[8], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "ndvi_winter",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[9], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "lst_summer",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[10], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "evi_summer",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[11], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "lst_spring",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[12], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "lst_fall",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[13], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "ndvi_spring",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[14], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "ndvi_summer",
    }
)
merged_df2 = merged_df2.drop(columns=["source_file"])
merged_df2 = pd.merge(merged_df2, landsat[15], on="GEOID", how="left")
merged_df2 = merged_df2.rename(
    columns={
        "mean": "evi_winter",
    }
)
PA_Landsat = merged_df2.drop(columns=["source_file"])

In [133]:
PA_Landsat.head()

,GEOID,lst_winter,ndvi_fall,evi_spring,evi_fall,savi_fall,savi_winter,savi_summer,savi_spring,ndvi_winter,lst_summer,evi_summer,lst_spring,lst_fall,ndvi_spring,ndvi_summer,evi_winter
0,42001030101,4.714395,0.672438,0.319802,0.410165,0.395006,0.263375,0.517445,0.316083,0.473559,33.257296,0.568514,10.827598,19.379386,0.501817,0.762120,0.260378
1,42001030103,4.470871,0.620192,0.291822,0.349453,0.341787,0.241329,0.491634,0.288043,0.434709,33.251842,0.537638,9.268735,19.461976,0.470976,0.736301,0.238932
2,42001030104,4.283530,0.616475,0.337012,0.368580,0.359570,0.279969,0.512783,0.328228,0.468789,33.716166,0.563929,10.559965,19.039489,0.509266,0.753435,0.280906
3,42001030200,3.399675,0.664453,0.351469,0.406248,0.392448,0.260190,0.522249,0.343635,0.459103,33.166265,0.575393,9.763646,18.368969,0.526715,0.764207,0.256791
4,42001030300,2.016297,0.706754,0.317563,0.415473,0.408640,0.249940,0.566242,0.317692,0.500742,30.361017,0.629287,10.687916,16.265277,0.535982,0.821622,0.239779


In [134]:
# everthing combined
PA_Retailers['GEOID'] = PA_Retailers['GEOID'].astype(str)
PA_Landsat['GEOID'] = PA_Landsat['GEOID'].astype(str)
PA_Landcover['GEOID'] = PA_Landcover['geoid'].astype(str)
PA_Census['GEOID'] = PA_Census['GEOID'].astype(str)
PA_Chronic['GEOID'] = PA_Chronic['LocationName'].astype(str)
PA_HRB['GEOID'] = PA_HRB['LocationName'].astype(str)

PA_Final = pd.merge(PA_Chronic, PA_HRB[["GEOID", "Smoking", "Drinking", "Short_Sleep", "Physical_Activity"]], on="GEOID", how="inner")
PA_Final = pd.merge(PA_Final, PA_Retailers[["GEOID", "neighbor_avg_density"]],  on="GEOID", how="inner")
PA_Final = pd.merge(PA_Final, PA_Census[["GEOID", "minority", "aging", "disability"]],  on="GEOID", how="inner")
PA_Final = pd.merge(PA_Final, PA_Landcover.drop(columns=["geoid"]),  on="GEOID", how="inner")
PA_Final = pd.merge(PA_Final, PA_Landsat,  on="GEOID", how="inner")
PA_Final = PA_Final.fillna(0)

In [135]:
PA_Final.head(5)

,CountyName,LocationName,Asthma,TotalPopulation,TotalPop18plus,COP,GEOID,Smoking,Drinking,Short_Sleep,...,savi_summer,savi_spring,ndvi_winter,lst_summer,evi_summer,lst_spring,lst_fall,ndvi_spring,ndvi_summer,evi_winter
0,Allegheny,42003141200,9.7,4007,3242,4.8,42003141200,10.3,21.3,32.1,...,0.342207,0.274674,0.316698,38.621956,0.359755,27.566184,23.110598,0.469827,0.569421,0.118931
1,Allegheny,42003140100,10.6,5579,5066,4.2,42003140100,10.7,23.9,34.5,...,0.365760,0.297535,0.347601,37.710135,0.397444,27.806910,21.461009,0.488617,0.570255,0.155803
2,Allegheny,42003191900,10.6,2177,1786,5.9,42003191900,14.6,22.1,35.0,...,0.307345,0.272597,0.277480,36.410736,0.324332,32.850463,21.101579,0.446655,0.489839,0.135087
3,Allegheny,42003320700,10.7,1613,1318,7.3,42003320700,16.4,21.3,36.0,...,0.430196,0.350588,0.367408,36.050876,0.472156,28.288317,19.085399,0.565722,0.654718,0.135720
4,Allegheny,42003100500,11.3,2013,1695,7.7,42003100500,15.3,15.8,40.6,...,0.414667,0.330390,0.356294,36.341476,0.456215,30.983730,20.809972,0.538620,0.632465,0.161533


In [136]:
# join with geometry if needed
PA_Final_Geom = PA_Final.merge(PA_Tracts[["GEOID", "geometry"]], on="GEOID", how="left")
PA_Final_Geom = gpd.GeoDataFrame(PA_Final_Geom, geometry='geometry')

In [137]:
PA_Final.to_csv('/content/drive/MyDrive/PA_Final.csv', index=False)
PA_Final_Geom.to_file('/content/drive/MyDrive/PA_Final_Geom.geojson', driver='GeoJSON')